In [2]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

## Split the Data into Training and Testing Sets

### Step 1: Read the `all_comments_cleaned.json` data from the `vinny` folder into a Pandas DataFrame.

In [19]:
# Read the JSON file from vinny folder into a Pandas DataFrame
youtube_comments_df = pd.read_json(
    Path('280_spam_dropped.json'),
)


# Review the DataFrame
youtube_comments_df = youtube_comments_df[youtube_comments_df['score']!='']
youtube_comments_df['score'].unique()


array([1, 2, 0], dtype=object)

In [20]:
youtube_comments_score = youtube_comments_df.drop (columns= ['comment', 'category', 'channel', 'video_url', 'video_id', 'channel_size', 'identity', 'results'])
youtube_comments_score = pd.to_numeric(youtube_comments_score['score'])
youtube_comments_score.shape

(82153,)

In [21]:
# Drop unnecessary columns
youtube_comments_dropped = youtube_comments_df.drop (columns= ['comment', 'channel', 'video_url', 'video_id', 'score'])
youtube_comments_dropped.shape

(82153, 4)

In [22]:
comments_numeric = pd.get_dummies(youtube_comments_dropped)
comments_numeric.info

<bound method DataFrame.info of         channel_size  identity  results  category_Automobiles  \
0                  0         1        0                     1   
1                  0         1        0                     1   
2                  0         1        0                     1   
3                  0         1        0                     1   
4                  0         1        0                     1   
...              ...       ...      ...                   ...   
189757             0         0        0                     0   
189758             0         0        0                     0   
189759             0         0        0                     0   
189760             0         0        0                     0   
189761             0         0        0                     0   

        category_Comedy  category_Education  category_Entertainment  \
0                     0                   0                       0   
1                     0                   0  

In [23]:
youtube_comments_joined = pd.concat([youtube_comments_score, comments_numeric], axis=1)
youtube_comments_joined.shape

(82153, 21)

### Step 2: Create the labels set (`y`)  from the “scores” column, and then create the features (`X`) DataFrame from the remaining columns.

In [24]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = youtube_comments_joined['score']
print(y.info())

# Separate the X variable, the features
X = youtube_comments_joined.drop(columns='score')

<class 'pandas.core.series.Series'>
Int64Index: 82153 entries, 0 to 189761
Series name: score
Non-Null Count  Dtype
--------------  -----
82153 non-null  int64
dtypes: int64(1)
memory usage: 1.3 MB
None


In [25]:
# Review the y variable Series
y[:5]

0    1
1    2
2    1
3    1
4    2
Name: score, dtype: int64

In [26]:
# Review the X variable DataFrame
X.head()
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82153 entries, 0 to 189761
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype
---  ------                           --------------  -----
 0   channel_size                     82153 non-null  int64
 1   identity                         82153 non-null  int64
 2   results                          82153 non-null  int64
 3   category_Automobiles             82153 non-null  uint8
 4   category_Comedy                  82153 non-null  uint8
 5   category_Education               82153 non-null  uint8
 6   category_Entertainment           82153 non-null  uint8
 7   category_Film                    82153 non-null  uint8
 8   category_Gaming                  82153 non-null  uint8
 9   category_Howto                   82153 non-null  uint8
 10  category_Music                   82153 non-null  uint8
 11  category_News and Politics       82153 non-null  uint8
 12  category_Nonprofit               82153 non-nu

### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [27]:
# Check the balance of our target values
y.value_counts()

1    31852
2    30616
0    19685
Name: score, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [28]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [29]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)
# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [30]:
# Score the model
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.44381147141883337
Testing Data Score: 0.44495837187789083


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [31]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({'Prediction': predictions, 'Actual': y_test})

,Prediction,Actual
139948,1,1
13717,0,0
164319,2,0
79142,1,2
59259,1,1
...,...,...
160180,1,1
3633,2,0
1887,2,1
20029,1,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [32]:
# Print the balanced_accuracy score of the model
accuracy_score = balanced_accuracy_score(y_test, predictions)
print(accuracy_score)

0.40865696769580534


In [33]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[ 722, 2390, 1903],
       [ 426, 4645, 2921],
       [ 493, 3267, 3772]])

In [34]:
# Print the classification report for the model
target_names = ["healthy loan", "high-risk loan"]
print(classification_report(y_test, predictions, target_names=target_names))

ValueError: Number of classes, 3, does not match size of target_names, 2. Try specifying the labels parameter